In [ ]:
!pip install accelerate -U
!pip install evaluate
!pip install seqeval


In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/100035.ann         
  inflating: data/100035.txt         
  inflating: data/100039.ann         
  inflating: data/100039.txt         
  inflating: data/100187.ann         
  inflating: data/100187.txt         
  inflating: data/100229.ann         
  inflating: data/100229.txt         
  inflating: data/100564.ann         
  inflating: data/100564.txt         
  inflating: data/100579.ann         
  inflating: data/100579.txt         
  inflating: data/100590.ann         
  inflating: data/100590.txt         
  inflating: data/100677.ann         
  inflating: data/100677.txt         
  inflating: data/100847.ann         
  inflating: data/100847.txt         
  inflating: data/100883.ann         
  inflating: data/100883.txt         
  inflating: data/100922.ann         
  inflating: data/100922.txt         
  inflating: data/101092.ann         
  inflating: data/101092.txt         
  inflating: data/101136.ann         
  inflating:

In [ ]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
import evaluate
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

def read_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read()

def parse_ann(ann_content):
    annotations = []
    for line in ann_content.strip().split('\n'):
        if line.startswith('T'):
            parts = line.split('\t')
            ann_id = parts[0]
            label_info = parts[1]
            text = parts[2]
            label_info_parts = label_info.split()
            label = label_info_parts[0]
            start = int(label_info_parts[1].split(';')[0])
            end = int(label_info_parts[2].split(';')[0])
            annotations.append({
                'id': ann_id,
                'label': label,
                'start': start,
                'end': end,
                'text': text
            })
    return annotations

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token not in string.punctuation]
    return filtered_tokens

def format_biobert_input(text, annotations):
    tokens = preprocess_text(text)
    token_annotations = ['O'] * len(tokens)
    text_offset = 0

    for ann in annotations:
        ann_tokens = word_tokenize(ann['text'])
        ann_label = ann['label']

        while text_offset < len(tokens):
            try:
                if tokens[text_offset] == ann_tokens[0]:
                    match = True
                    for i in range(len(ann_tokens)):
                        if text_offset + i >= len(tokens) or tokens[text_offset + i] != ann_tokens[i]:
                            match = False
                            break
                    if match:
                        for i in range(len(ann_tokens)):
                            if i == 0:
                                token_annotations[text_offset + i] = f'B-{ann_label}'
                            else:
                                token_annotations[text_offset + i] = f'I-{ann_label}'
                        text_offset += len(ann_tokens)
                        break
                text_offset += 1
            except:
                print(ann_tokens)
                print(tokens[text_offset])

    return tokens, token_annotations

def split_into_chunks(tokens, labels, max_length=509):
    chunks = []
    chunk_labels = []
    current_chunk = []
    current_chunk_labels = []
    current_length = 0

    for i in range(len(tokens)):
        current_chunk.append(tokens[i])
        current_chunk_labels.append(labels[i])
        current_length += 1

        if current_length >= max_length:
            while i < len(tokens) and not labels[i] == 'O':
                current_chunk.append(tokens[i])
                current_chunk_labels.append(labels[i])
                current_length += 1
                i += 1

            chunks.append(current_chunk)
            chunk_labels.append(current_chunk_labels)
            current_chunk = []
            current_chunk_labels = []
            current_length = 0

    if current_chunk:
        chunks.append(current_chunk)
        chunk_labels.append(current_chunk_labels)

    return chunks, chunk_labels

def process_files(txt_file, ann_file):
    text = read_file(txt_file)
    ann_content = read_file(ann_file)
    annotations = parse_ann(ann_content)
    tokens, labels = format_biobert_input(text, annotations)
    token_chunks, label_chunks = split_into_chunks(tokens, labels)
    return token_chunks, label_chunks

tokend_text = []
cor_labels = []

def process_all_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            txt_file = os.path.join(directory, filename)
            ann_file = txt_file.replace(".txt", ".ann")
            if os.path.exists(ann_file):
                token_chunks, label_chunks = process_files(txt_file, ann_file)
                tokend_text.extend(token_chunks)
                cor_labels.extend(label_chunks)

directory = '/content/data'
process_all_files(directory)

labels = ["O", "B-Drug", "I-Drug", "B-Strength", "I-Strength", "B-Form", "I-Form", "B-Dosage", "I-Dosage",
          "B-Duration", "I-Duration", "B-Frequency", "I-Frequency", "B-Route", "I-Route", "B-ADE", "I-ADE",
          "B-Reason", "I-Reason"]

label_map = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label_map.items()}

tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
model = AutoModelForTokenClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=len(id2label), id2label=id2label, label2id=label_map, ignore_mismatched_sizes=False)

texts = tokend_text
labels = [[label_map[label] for label in doc_labels] for doc_labels in cor_labels]

class NERDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        word_labels = self.labels[index]

        encoding = self.tokenizer(text,
                                  truncation=True,
                                  padding='max_length',
                                  max_length=self.max_len,
                                  is_split_into_words=True,
                                  return_tensors='pt')

        word_ids = encoding.word_ids(batch_index=0)

        labels = [-100 if word_id is None else word_labels[word_id] for word_id in word_ids]

        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(labels, dtype=torch.long)

        return item

MAX_LEN = 512
BATCH_SIZE = 16

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.1)

kf = KFold(n_splits=10, shuffle=True, random_state=42)
all_metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_micro': [],
    'f1_macro': [],
    'confusion_matrix': []
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="no"
)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1_micro": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

for fold, (train_index, val_index) in enumerate(kf.split(train_texts)):
    print(f"Fold {fold + 1}")

    train_fold_texts = [train_texts[i] for i in train_index]
    val_fold_texts = [train_texts[i] for i in val_index]
    train_fold_labels = [train_labels[i] for i in train_index]
    val_fold_labels = [train_labels[i] for i in val_index]

    train_dataset = NERDataset(train_fold_texts, train_fold_labels, tokenizer, MAX_LEN)
    val_dataset = NERDataset(val_fold_texts, val_fold_labels, tokenizer, MAX_LEN)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    metrics = trainer.evaluate(val_dataset)
    predictions, labels, _ = trainer.predict(val_dataset)
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []

    for i in range(len(labels)):
        true_labels.extend(labels[i])
        pred_labels.extend(predictions[i])

    true_labels = [label for label in true_labels if label != -100]
    pred_labels = [pred for label, pred in zip(true_labels, pred_labels) if label != -100]

    cm = confusion_matrix(true_labels, pred_labels, labels=list(label_map.values()))

    all_metrics['accuracy'].append(metrics['eval_accuracy'])
    all_metrics['precision'].append(metrics['eval_precision'])
    all_metrics['recall'].append(metrics['eval_recall'])
    all_metrics['f1_micro'].append(metrics['eval_f1_micro'])
    all_metrics['f1_macro'].append(precision_recall_fscore_support(true_labels, pred_labels, average='macro')[2])
    all_metrics['confusion_matrix'].append(cm)

    print(f"Fold {fold + 1} Metrics:")
    print(f"Accuracy: {metrics['eval_accuracy']}")
    print(f"Precision: {metrics['eval_precision']}")
    print(f"Recall: {metrics['eval_recall']}")
    print(f"Micro F1: {metrics['eval_f1_micro']}")
    print(f"Macro F1: {all_metrics['f1_macro'][-1]}")


confusion_matrices = np.sum(all_metrics['confusion_matrix'], axis=0)

metrics_mean_std = {metric: (np.mean(all_metrics[metric]), np.std(all_metrics[metric])) for metric in all_metrics if metric != 'confusion_matrix'}

print("\nMetrics Mean and Standard Deviation:")
for metric, (mean, std) in metrics_mean_std.items():
    print(f"{metric.capitalize()} - Mean: {mean}, Std: {std}")

print("\nCombined Confusion Matrix:")
print(confusion_matrices)

trainer.save_model("./model")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Fold 8 Metrics:
Accuracy: 0.9999027308318945
Precision: 0.9835164835164835
Recall: 0.988950276243094
Micro F1: 0.9862258953168045
Macro F1: 0.10797994141115101
Fold 9


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Micro,Accuracy
1,0.000500,0.000359,0.988571,1.000000,0.994253,0.999898
2,0.000800,0.000175,0.994253,1.000000,0.997118,0.999949
3,0.000100,0.000048,1.000000,1.000000,1.000000,1.000000
4,0.000100,0.000024,1.000000,1.000000,1.000000,1.000000
5,0.000100,0.000010,1.000000,1.000000,1.000000,1.000000
6,0.000100,0.000011,1.000000,1.000000,1.000000,1.000000
7,0.000100,0.000008,1.000000,1.000000,1.000000,1.000000
8,0.000100,0.000015,1.000000,1.000000,1.000000,1.000000
9,0.000100,0.000048,1.000000,1.000000,1.000000,1.000000
10,0.006900,0.003427,0.932203,0.953757,0.942857,0.999465


Fold 9 Metrics:
Accuracy: 0.9994653495595499
Precision: 0.9322033898305084
Recall: 0.953757225433526
Micro F1: 0.9428571428571428
Macro F1: 0.10197136090226915
Fold 10


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Micro,Accuracy
1,0.002500,0.002269,0.909396,1.000000,0.952548,0.999325
2,0.001300,0.001040,0.960993,1.000000,0.980108,0.999725
3,0.000500,0.001031,0.954225,1.000000,0.976577,0.999675
4,0.000200,0.000876,0.954225,1.000000,0.976577,0.999675
5,0.000100,0.000575,0.967857,1.000000,0.983666,0.999775
6,0.000400,0.001593,0.931271,1.000000,0.964413,0.999500
7,0.000500,0.001351,0.978339,1.000000,0.989051,0.999850
8,0.000400,0.001151,0.954225,1.000000,0.976577,0.999675
9,0.000700,0.001133,0.940767,0.996310,0.967742,0.999600
10,0.000300,0.000854,0.957597,1.000000,0.978339,0.999700


Fold 10 Metrics:
Accuracy: 0.9996998499249625
Precision: 0.9575971731448764
Recall: 1.0
Micro F1: 0.9783393501805053
Macro F1: 0.07180481551604348

Metrics Mean and Standard Deviation:
Accuracy - Mean: 0.9973641113483133, Std: 0.003370671919924848
Precision - Mean: 0.8227075232571543, Std: 0.21684646706920616
Recall - Mean: 0.8494365859350452, Std: 0.2783059484104994
F1_micro - Mean: 0.8275458877516243, Std: 0.26679791043321444
F1_macro - Mean: 0.08455019608888985, Std: 0.019765380285472322

Combined Confusion Matrix:
[[398446   1893     67    266    124     54      0     38     10      8
       0    131     19    122      0     38      7    199    166]
 [  2074    120      0      6      0      0      0      1      2      0
       0      2      0      3      0      0      0      0      3]
 [    78      8      4      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0]
 [   291     16      4      3      0      0      0      0     

In [ ]:
import numpy as np
import evaluate
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support

metric = evaluate.load("seqeval")

def compute_test_metrics(predictions, labels):
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []

    for i in range(len(labels)):
        true_labels.append(labels[i])
        pred_labels.append(predictions[i])

    true_labels = [[label for label in sublist if label != -100] for sublist in true_labels]
    pred_labels = [[pred for pred, label in zip(pred_sublist, label_sublist) if label != -100]
                   for pred_sublist, label_sublist in zip(pred_labels, true_labels)]

    flat_true_labels = [item for sublist in true_labels for item in sublist]
    flat_pred_labels = [item for sublist in pred_labels for item in sublist]

    cm = confusion_matrix(flat_true_labels, flat_pred_labels, labels=list(label_map.values()))

    results = metric.compute(predictions=[[id2label[p] for p in sublist] for sublist in pred_labels],
                             references=[[id2label[l] for l in sublist] for sublist in true_labels])

    precision, recall, f1_weighted, _ = precision_recall_fscore_support(
        flat_true_labels, flat_pred_labels, average='weighted', zero_division=0
    )
    _, _, f1_micro, _ = precision_recall_fscore_support(
        flat_true_labels, flat_pred_labels, average='micro', zero_division=0
    )
    _, _, f1_macro, _ = precision_recall_fscore_support(
        flat_true_labels, flat_pred_labels, average='macro', zero_division=0
    )

    return {
        "confusion_matrix": cm,
        "accuracy": accuracy_score(flat_true_labels, flat_pred_labels),
        "precision": precision,
        "recall": recall,
        "f1_micro": f1_micro,
        "f1_macro": f1_macro
    }

test_dataset = NERDataset(test_texts, test_labels, tokenizer, MAX_LEN)

predictions, labels, _ = trainer.predict(test_dataset)

test_metrics = compute_test_metrics(predictions, labels)

for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value}")


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


confusion_matrix: [[45811    64     5     7     4     0     0     0     0     0     0     1
      0     0     0     8     0     9     3]
 [  262     5     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [   10     1     1     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [   45     1     0     4     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [   25     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    8     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    7     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    1     0     0     0     

In [ ]:
import evaluate

# Load evaluation metric
metric = evaluate.load("seqeval")

# Compute metrics for test dataset
def compute_test_metrics(predictions, labels):
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []

    for i in range(len(labels)):
        true_labels.append(labels[i])
        pred_labels.append(predictions[i])

    # Remove ignored index (-100)
    true_labels = [[label for label in sublist if label != -100] for sublist in true_labels]
    pred_labels = [[pred for pred, label in zip(pred_sublist, label_sublist) if label != -100]
                   for pred_sublist, label_sublist in zip(pred_labels, true_labels)]

    # Flatten the lists for confusion matrix
    flat_true_labels = [item for sublist in true_labels for item in sublist]
    flat_pred_labels = [item for sublist in pred_labels for item in sublist]

    # Calculate confusion matrix
    cm = confusion_matrix(flat_true_labels, flat_pred_labels, labels=list(label_map.values()))

    results = metric.compute(predictions=[[id2label[p] for p in sublist] for sublist in pred_labels],
                             references=[[id2label[l] for l in sublist] for sublist in true_labels])

    return {
        "confusion_matrix": cm,
        "accuracy": accuracy_score(flat_true_labels, flat_pred_labels),
        "precision": precision_recall_fscore_support(flat_true_labels, flat_pred_labels, average='weighted')[0],
        "recall": precision_recall_fscore_support(flat_true_labels, flat_pred_labels, average='weighted')[1],
        "f1_micro": precision_recall_fscore_support(flat_true_labels, flat_pred_labels, average='micro')[2],
        "f1_macro": precision_recall_fscore_support(flat_true_labels, flat_pred_labels, average='macro')[2]
    }

# Test dataset preparation
test_dataset = NERDataset(test_texts, test_labels, tokenizer, MAX_LEN)

# Predict on test dataset
predictions, labels, _ = trainer.predict(test_dataset)

# Compute metrics
test_metrics = compute_test_metrics(predictions, labels)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r model/ drive/MyDrive